# Get Training Data

In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/root/fruitao/drone/logic_guided_prompt/code /root/fruitao/drone/logic_guided_prompt


In [2]:
from logic.description_logic import *
from utils.load_sensor_data import *
from data_augmentation.text_augmentation import *

/root/fruitao/drone/logic_guided_prompt/code/data_augmentation /root/fruitao/drone/logic_guided_prompt/code


In [3]:
# old_format_example=pd.ExcelFile(r"../text_annotation/rural.xlsx")

dataset_file_li=[r"../../text_annotation/desert.xlsx", r"../../text_annotation/disturbance.xlsx", r"../../text_annotation/factory.xlsx", r"../../text_annotation/island.xlsx", 
                 r"../../text_annotation/misc.xlsx", r"../../text_annotation/ocean.xlsx", r"../../text_annotation/rural.xlsx", r"../../text_annotation/urban.xlsx"
]

old_data_dic=dict()

for dataset_file in dataset_file_li:
    print("----processing file: {}----".format(dataset_file))
    
    old_format_example=pd.ExcelFile(dataset_file)

    ts_converters={'name': str, 'Type': str, 'Moving': str2bool, 'InPath': str2bool, 'time_stamp': str}
    status_converters={'PilotExperienced': str2bool, 'Low_visibility': str2bool, 'Normal_frame': str2bool,
                      'weather': str, 'upside_down': str2bool, 'good_motor_condition': str2bool,
                      'going_backwards': str2bool, 'indoor': str2bool, 'waterproof drone?': str2bool,
                      'flying over': str, 'criticality': str, 
                      'RiskOfPhysicalDamage': str, 'RiskOfInternalDamage': str2bool,
                      'RiskOfHumanDamage': str2bool, 'LostConnection': str2bool}
    text_converters={'Link':str, 'Text1':str, 'Text2':str}

    old_format_ts=pd.read_excel(old_format_example, sheet_name='timestep', converters=ts_converters)
    old_format_status=pd.read_excel(old_format_example, sheet_name='status', converters=status_converters)
    old_format_text=pd.read_excel(old_format_example, sheet_name='text', converters=text_converters)

    print("---loading sensor data---")
    load_ts_to_dict(old_format_ts, old_data_dic)
    load_status_to_dict(old_format_status, old_data_dic)
    load_text_to_dict(old_format_text, old_data_dic)

    # print("---generating templates---")
    for link, data in old_data_dic.items():
        templates, related_status_dic, related_timestep_dic = gen_templates(data)
        old_data_dic[link]['templates'] = templates
        old_data_dic[link]['related_status_dic'] = related_status_dic
        old_data_dic[link]['related_timestep_dic'] = related_timestep_dic


----processing file: ../../text_annotation/desert.xlsx----
---loading sensor data---
templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m."], 'reachable_inpath': ["(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]}
templates:{'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:03) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.2m."], 'reachable_inpath': ["(0:03) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.2m."]}
templates:{'flying_not_normal_frame

In [6]:
import json

link_li=[]
text_li=[]
dl_name_li=[]
templates_li=[]
related_sensor_data_li=[]
related_status_dic_li=[]
related_timestep_dic_li=[]
dl_related_sensor_data_mapping_li=[]

# templates_nested_li=[]
# related_status_dic_nested_li=[]
# related_timestep_dic_nested_li=[]
# related_sensor_data_nested_li=[]

# related_sensor_data_li as input and templates_li as label
for link, value in old_data_dic.items():
#     print("value:{}".format(value))
    link_li.append(link)
    if 'text' not in value:
        text_li.append([])
    else:    
        text_li.append(value['text'])
    dl_index=0
    
#     dl_related_sensor_data_mapping={}
#     related_status_dic_li=[]
#     related_timestep_dic_li=[]
#     related_sensor_data_li=[]
#     templates_li=[]
    for dl_name, template_value in value['templates'].items():
        templates_li.extend(template_value)
        if dl_name in value['related_status_dic']:
            related_status_dic_li.extend(value['related_status_dic'][dl_name])
            related_sensor_data_li.extend(value['related_status_dic'][dl_name])
        elif dl_name in value['related_timestep_dic']:
            related_timestep_dic_li.extend(value['related_timestep_dic'][dl_name])
            related_sensor_data_li.extend(value['related_timestep_dic'][dl_name])
        else:
            print("WHAT?")
        
related_sensor_data_li=[json.dumps(dic) for dic in related_sensor_data_li]
#         dl_related_sensor_data_mapping[dl_name]=[i+dl_index for i in range(len(template_value))]
#         dl_index+=len(template_value)
        
#     dl_related_sensor_data_mapping_li.append(dl_related_sensor_data_mapping)
#     related_status_dic_nested_li.append(related_status_dic_li)
#     related_timestep_dic_nested_li.append(related_timestep_dic_li)
#     templates_nested_li.append(templates_li)
#     related_sensor_data_nested_li.append(related_sensor_data_li)

In [12]:
import math

def train_test_split(all_li, train_ratio=0.8, val_ratio=0.1):
    all_len=len(all_li)
    train_len=math.ceil(train_ratio*all_len)
    val_len=math.ceil(val_ratio*all_len)
    
    return all_li[:train_len], all_li[train_len:train_len+val_len], all_li[train_len:train_len+val_len:]

train_sensor_data, val_sensor_data, test_sensor_data=train_test_split(related_sensor_data_li)
train_templates, val_templates, test_templates=train_test_split(templates_li)

print(len(related_sensor_data_li), len(train_sensor_data), len(val_sensor_data), len(test_sensor_data))
print(len(templates_li), len(train_templates), len(val_templates), len(test_templates))

1652 1322 166 166
1652 1322 166 166


In [13]:
# mapping{'flying_empty_battery':[0], 'flying_not_normal_frame':[1], 'reachable_inpath':[2, 3, 4]}
# input: [{'Battery_level': 0}, {'Normal_frame': False}, {'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name': 'gravestone 2', 'Distance': 3.0}, {'time_stamp': '0:05', 'name': 'castle ', 'Distance': 1.0}]
# output: 'The flying drone is runing out of battery with only 0% charge.'

print(len(related_sensor_data_li), related_sensor_data_li[0], type(related_sensor_data_li[0]))
print(len(templates_li), templates_li[0], type(templates_li[0]))
# print(dl_related_sensor_data_mapping_li[2])

1652 {"Normal_frame": false} <class 'str'>
1652 The drone is flying with a damaged frame. <class 'str'>


# Augmented Prompt

In [14]:
!nvidia-smi

Fri Jun  9 15:05:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   32C    P0    64W / 300W |  11691MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:22:00.0 Off |                    0 |
| N/A   

In [15]:
import torch

device=torch.device("cuda:3")

In [16]:
import math

from tqdm import tqdm
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.nn as nn
from sklearn.metrics import accuracy_score
from nltk.translate import meteor_score

/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
torch.manual_seed(3407)

In [18]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [19]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
initialize_from_vocab = True
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

/root/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [23]:
parameters = list(model.parameters())
for x in parameters[1:]:
    x.requires_grad = False

In [24]:
str_warning_dl_list=[dl_func.__name__ for dl_func in WARNING_DL]
print(str_warning_dl_list)

['out_of_range_low_battery', 'flying_empty_battery', 'empty_battery_water_proof_flying_water', 'flying_not_normal_frame', 'low_battery_low_temperature', 'low_battery_strong_wind', 'low_battery_high_altitude', 'water_proof_precipitation', 'flying_water_low_altitude', 'rain_strong_wind', 'upsidedown_inexperienced', 'extreme_weather_inexperienced', 'gloomy_high_altitude', 'indoor_near_human', 'low_visibility_near_object', 'low_altitude_fast_speed', 'near_fragile_object_near_human', 'near_object_inexperienced', 'near_moving_object', 'reachable_inpath', 'very_close_human', 'very_close_lamp', 'very_close_object']


In [ ]:
# # Todo: Use dataset instead.

# def generate_data(batch_size, n_tokens, related_sensor_data, text_data):

#     def yield_data(x_batch, y_batch, l_batch):
            
#         x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
#         y = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True)
# #         y = torch.cat(y_batch, dim=0)
            
#         # Todo: Add prompt mask for different description logic
#         m = (x > 0).to(torch.float32)
#         decoder_input_ids = torch.full((x.shape[0], y.shape[1]), 1)
        
#         if torch.cuda.is_available():
#             x = x.to(device)
#             y = y.to(device)
#             m = m.to(device)
#             decoder_input_ids = decoder_input_ids.to(device)
#         return x, y, m, decoder_input_ids, l_batch

#     x_batch, y_batch, l_batch = [], [], []
#     for x, y in zip(related_sensor_data, text_data):
        
#         context = x
#         inputs = tokenizer(context, return_tensors="pt")
#         inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        
#         outputs=tokenizer(y, return_tensors="pt")
# #         print(type(inputs), inputs['input_ids'][0].size(), inputs['input_ids'].size(), 
# #               type(outputs), outputs['input_ids'][0].size(), outputs['input_ids'].size())
        
#         outputs_ids = torch.cat([torch.full((1, n_tokens - 1), -100), outputs['input_ids']], 1)
        
#         x_batch.append(inputs['input_ids'][0])
#         y_batch.append(outputs_ids[0])
#         l_batch.append(outputs['input_ids'][0])
#         if len(x_batch) >= batch_size:
#             yield yield_data(x_batch, y_batch, l_batch)
#             x_batch, y_batch, l_batch = [], [], []

#     if len(x_batch) > 0:
#         yield yield_data(x_batch, y_batch, l_batch)
#         x_batch, y_batch, l_batch = [], [], []

In [29]:
# Todo: Use dataset instead.

def generate_data(batch_size, n_tokens, related_sensor_data, text_data):

    def yield_data(x_batch, y_batch, l_batch):
            
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
        y = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True)
#         y = torch.cat(y_batch, dim=0)
            
        # Todo: Add prompt mask for different description logic
        m = (x > 0).to(torch.float32)
        decoder_input_ids = torch.full((x.shape[0], y.shape[1]), 1)
        
        if torch.cuda.is_available():
            x = x.to(device)
            y = y.to(device)
            m = m.to(device)
            decoder_input_ids = decoder_input_ids.to(device)
        return x, y, m, decoder_input_ids, l_batch

    x_batch, y_batch, l_batch = [], [], []
    for x, y in zip(related_sensor_data, text_data):
        
        context = x
        inputs = tokenizer(context, return_tensors="pt")
        inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        
        outputs=tokenizer(y, return_tensors="pt")
#         print(type(inputs), inputs['input_ids'][0].size(), inputs['input_ids'].size(), 
#               type(outputs), outputs['input_ids'][0].size(), outputs['input_ids'].size())
        
        outputs_ids = torch.cat([torch.full((1, n_tokens - 1), -100), outputs['input_ids']], 1)
        
        x_batch.append(inputs['input_ids'][0])
        y_batch.append(outputs_ids[0])
        l_batch.append(outputs['input_ids'][0])
        if len(x_batch) >= batch_size:
            yield yield_data(x_batch, y_batch, l_batch)
            x_batch, y_batch, l_batch = [], [], []

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [ ]:
def __sensor_data_reader(li):
    for line in li:
        if multiple:
            yield [item.lower().split() for item in line.strip().split("\t")]
        else:
            yield line.strip().lower().split()

def __prediction_reader(li):
    
    for k, value in dic.items():
        result=''
        for kv, vv in value[key].items():
            for template in vv:
                result+=template
        # print("__dict_key_reader", result)
        yield result.strip().lower().split()

def __template_reader(li):
    for k, value in dic.items():       
        result=[]
        
        # print("__sensor_reader: value[{}]".format('timestep'), value['timestep'][0])
        for timestep in value['timestep']:
            for kv, vv in timestep.items():
                kv_tokens=str(kv).strip().lower().split()
                vv_tokens=str(vv).strip().lower().split()
                result.append([kv_tokens, vv_tokens])
    
        # print("__sensor_reader: value[{}]".format('status'), value['status'])
        for kv, vv in value['status'][0].items():
            kv_tokens=str(kv).strip().lower().split()
            vv_tokens=str(vv).strip().lower().split()
            result.append([kv_tokens, vv_tokens])
            
        # print("__sensor_reader", result)
        yield result
        
def __ref_reader(dic, key1, key2):
    for k, value in dic.items():
        # print("__dict_key_key_reader", value[key1][0][key2], type(value[key1][0][key2]))
        yield value[key1][0][key2].strip().lower().split()
            

In [43]:
# Test generation function
output_max_pred=[]

for x, y, m, dii, true_labels in generate_data(4, n_tokens, related_sensor_data_li, templates_li):
    print("related_sensor_data_li", related_sensor_data_li[:4])
    print("templates_li", templates_li[:4])
    
    assert dii.shape == y.shape
    outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    
    assert outputs['logits'].shape[:2] == y.shape
    max_pred_ids = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
    
    for i, max_pred_ids_unbatch in enumerate(max_pred_ids):
        
        max_pred_ids_unbatch=[ids for ids in max_pred_ids_unbatch if ids!=0]
        max_pred_ids[i]=max_pred_ids_unbatch
        
    output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
    print("output_max_pred:{}".format((output_max_pred)))
    
    inputs_it=__sensor_reader(related_sensor_data_li)
    prediction=__prediction_reader(outputs)
    outputs_it=__template_reader(templates_li)

    break

related_sensor_data_li ['{"Normal_frame": false}', '{"time_stamp": "0:01", "name": "car", "Distance": 3.0}', '{"time_stamp": "0:05", "name": "car", "Distance": 0.3}', '{"time_stamp": "0:05", "name": "car", "Distance": 0.3}']
templates_li ['The drone is flying with a damaged frame.', "(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m.", "(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]
output_max_pred:["'a.ss subjecta   to of fulle int- is not for’ The drone is flying with a damage frame</s></s>", "the  s..s thea is  to of fulle int- is not fore (0:01) The drone is facing the physical physical damage. It's flying flying flying car car car drone   flight distance of 3. distance ofm pilot</s></s></s>", 'the  0.3..s thea is  to of fulle int- is not for

In [ ]:
reference_it = __ref_reader(old_data_dic, "text", 'Text1')
generation_it = __gen_reader(old_data_dic, "templates")
table_it = __sensor_reader(old_data_dic)

entail_method=overlap_probability
precision, recall, f_score, all_f = parent(
    generation_it,
    reference_it,
    table_it,
    lambda_weight=None,
    smoothing=0.00001,
    entailment_fn=entail_method)

print("precision", precision)
print("recall", recall)
print("f_score", f_score)
print("all_f", all_f)

NameError: name 'overlap_probability' is not defined

In [ ]:
batch_size = 32
n_epoch = 300
# total_batch = math.ceil(len(related_sensor_data_li) / batch_size)
# dev_total_batch = math.ceil(len(templates_li) / batch_size)
total_batch = math.ceil(len(train_sensor_data) / batch_size)
dev_total_batch = math.ceil(len(val_sensor_data) / batch_size)
use_ce_loss = False
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

test_losses_epoch=[]
bleu_score_epoch=[]
rouge_score_epoch=[]
meteor_score_epoch=[]

from datasets import load_metric

bleu=load_metric("bleu")
rouge=load_metric("rouge")
meteor = load_metric('meteor')

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    train_losses = []
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, train_sensor_data, train_templates)), total=total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        
        optimizer.zero_grad()
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels

        if use_ce_loss:
            logits = outputs['logits'][:, :, :]
            true_labels_tensor = torch.tensor(true_labels, dtype=torch.long).cuda()
            loss = ce_loss(logits, true_labels_tensor)
        else:
            loss = outputs.loss
        loss.backward()
        optimizer.step()
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        train_losses.append(loss_value)

#         acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(train_losses):.4f}')

    all_true_labels = []
    all_pred_labels = []
    output_max_pred=[]
    test_losses = []
    
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, val_sensor_data, val_templates)), total=dev_total_batch)
        for i, (x, y, m, dii, true_labels) in pbar:
            all_true_labels += true_labels
            outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            
            output_ids=outputs.logits[:, n_tokens:, :]
            max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
            max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
            output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
            
            loss = outputs.loss
            loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
            test_losses.append(loss_value)
            
            pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
            all_pred_labels += pred_labels
            pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')
            
    bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
    bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]
    
    meteor_output = [meteor_score.single_meteor_score(
        tokenizer.tokenize(ref_s), tokenizer.tokenize(pred_s), alpha=0.9, beta=3, gamma=0.5)
                     for ref_s, pred_s in zip(output_max_pred, test_templates)
                    ]
    meteor_avg_score=sum(meteor_output) / len(meteor_output)
    
    bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
    rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
    
    bleu_score_epoch.append(bleu_score)
    rouge_score_epoch.append(rouge_score)
    meteor_score_epoch.append(meteor_avg_score)
    
    test_losses_epoch.append(np.mean(test_losses))

<ipython-input-27-6e2f61e46518>:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu=load_metric("bleu")
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


epoch 0


dev: loss=0.2099: 100%|██████████| 6/6 [00:00<00:00,  8.75it/s]


epoch 1


dev: loss=0.1219: 100%|██████████| 6/6 [00:00<00:00,  8.81it/s]


...

epoch 72


dev: loss=0.0343: 100%|██████████| 6/6 [00:00<00:00,  8.33it/s]


epoch 73


train: loss=0.0281:  21%|██▏       | 9/42 [00:01<00:05,  6.44it/s]


KeyboardInterrupt: 

In [ ]:
print("test_sensor_data: ", len(test_sensor_data))
print("output_max_pred:", output_max_pred[0], len(output_max_pred))


In [ ]:
rouge1_precision_epoch=[round(rouge_output["rouge1"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge1_recall_epoch=[round(rouge_output["rouge1"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge1_fmeasure_epoch=[round(rouge_output["rouge1"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rouge2_precision_epoch=[round(rouge_output["rouge2"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge2_recall_epoch=[round(rouge_output["rouge2"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge2_fmeasure_epoch=[round(rouge_output["rouge2"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeL_precision_epoch=[round(rouge_output["rougeL"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeL_recall_epoch=[round(rouge_output["rougeL"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeL_fmeasure_epoch=[round(rouge_output["rougeL"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeLsum_precision_epoch=[round(rouge_output["rougeLsum"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeLsum_recall_epoch=[round(rouge_output["rougeLsum"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeLsum_fmeasure_epoch=[round(rouge_output["rougeLsum"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

            
bleu_avg_score_epoch= [round(bleu_output["bleu"], 4) for bleu_output in bleu_score_epoch]
bleu_1_score_epoch=[round(bleu_output["precisions"][0], 4) for bleu_output in bleu_score_epoch]
bleu_2_score_epoch=[round(bleu_output["precisions"][1], 4) for bleu_output in bleu_score_epoch]
bleu_3_score_epoch=[round(bleu_output["precisions"][2], 4) for bleu_output in bleu_score_epoch]
bleu_4_score_epoch=[round(bleu_output["precisions"][3], 4) for bleu_output in bleu_score_epoch]

meteor_score_epoch=[round(meteor_avg, 4) for meteor_avg in meteor_score_epoch]

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'rouge1_precision_epoch': rouge1_precision_epoch,
    'rouge1_recall_epoch': rouge1_recall_epoch,
    'rouge1_fmeasure_epoch': rouge1_fmeasure_epoch,

    'rouge2_precision_epoch': rouge2_precision_epoch,
    'rouge2_recall_epoch': rouge2_recall_epoch,
    'rouge2_fmeasure_epoch': rouge2_fmeasure_epoch,

    'rougeL_precision_epoch': rougeL_precision_epoch,
    'rougeL_recall_epoch': rougeL_recall_epoch,
    'rougeL_fmeasure_epoch': rougeL_fmeasure_epoch,

    'rougeLsum_precision_epoch': rougeLsum_precision_epoch,
    'rougeLsum_recall_epoch': rougeLsum_recall_epoch,
    'rougeLsum_fmeasure_epoch': rougeLsum_fmeasure_epoch,

    'bleu_avg_score_epoch': bleu_avg_score_epoch,
    'bleu_1_score_epoch': bleu_1_score_epoch,
    'bleu_2_score_epoch': bleu_2_score_epoch,
    'bleu_3_score_epoch': bleu_3_score_epoch,
    'bleu_4_score_epoch': bleu_4_score_epoch,
    
    'meteor_score_epoch': meteor_score_epoch,
})

df.to_csv('metrics_epoch_soft_prompt_3407.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

plt.plot(test_losses_epoch, '*-r')

plt.plot(rouge1_precision_epoch, '*--g')
plt.plot(rouge1_recall_epoch, 'x--g')
plt.plot(rouge1_fmeasure_epoch, 'o--g')

plt.plot(bleu_avg_score_epoch, '*:b')
plt.plot(bleu_1_score_epoch, 'x:b')
plt.plot(bleu_2_score_epoch, '+:b')
plt.plot(bleu_3_score_epoch, 'o:b')
plt.plot(bleu_4_score_epoch, 'd:b')

plt.plot(meteor_score_epoch, '*-.y')

plt.show()

# Test

In [ ]:
print(123)

In [ ]:
test_total_batch = math.ceil(len(test_sensor_data) / batch_size)

with torch.no_grad():
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            
        output_ids=outputs.logits[:, n_tokens:, :]
        max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
        max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
        output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
            
        loss = outputs.loss
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        test_losses.append(loss_value)
            
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels
        pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')
            
    bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
    bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]
    
    meteor_output = [meteor_score.single_meteor_score(
        tokenizer.tokenize(ref_s), tokenizer.tokenize(pred_s), alpha=0.9, beta=3, gamma=0.5)
                     for ref_s, pred_s in zip(output_max_pred, test_templates)
                    ]
    meteor_avg_score=sum(meteor_output) / len(meteor_output)
    
    bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
    rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)

print("test bleu", bleu_score)
print("test rouge", rouge_score)
print("test meteor", meteor_avg_score)

In [57]:
inputs = tokenizer(test_sensor_data[0], return_tensors="pt").to(device)
print(test_sensor_data[0])

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1).to(device), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1).to(device), inputs['attention_mask']], 1)
inputs['decoder_input_ids']=torch.full((1, 100), 1).to(device)
 
outputs = model(**inputs)
print(outputs.logits.shape)
output_ids=outputs.logits[:, n_tokens:, :]
print(output_ids.shape)
max_ids=output_ids.argmax(-1).detach().cpu().numpy().tolist()
# max_ids[0]=[ids for ids in max_ids[0] if ids!=0]
print(tokenizer.batch_decode(max_ids))

{"time_stamp": "0:05", "name": NaN, "Distance": 0.5}
torch.Size([1, 100, 32128])
torch.Size([1, 77, 32128])
[":05) The drone is facing the risk of physical damage. It's flying too close to nan at a distance of 0.5m, and the pilot is not<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>."]


In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m_%H%M%S")
torch.save(model.state_dict(), "./output/T5_"+dt_string)

In [ ]:
model = MyModel(num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
output_model = './models/model_xlnet_mid.pth'

# save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

# load
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.load_state_dict(torch.load("./output/T5_2404_092246"))
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

In [ ]:
from datasets import load_metric

In [ ]:
all_true_labels = []
all_pred_labels = []
output_max_pred=[]
test_losses = []
with torch.no_grad():
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        
        output_ids=outputs.logits[:, n_tokens:, :]
        max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
        max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
        output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
        
        loss = outputs.loss
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        test_losses.append(loss_value)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels
        pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')

In [ ]:
print(output_max_pred)

In [ ]:
bleu=load_metric("bleu")
rouge=load_metric("rouge")

In [ ]:
bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]


In [ ]:
bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
print(bleu_score)
rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
print(rouge_score)

In [ ]:
tokenizer.encode("Hello world")